In [1]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.5.5_linear_scale-up_etc/ciff_sam/2021_11_23_17_59_09/count_data/'

In [3]:
wasting_pt = pd.read_csv(output_dir +'wasting_state_person_time.csv')
wasting_pt.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,x_factor,sq_lns,mam_treatment,sam_treatment,age
0,0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,covered,covered,early_neonatal
1,1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,covered,covered,covered,early_neonatal
2,2,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,uncovered,covered,early_neonatal
3,3,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,covered,uncovered,covered,early_neonatal
4,4,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,covered,uncovered,early_neonatal


In [4]:
wasting_transitions = pd.read_csv(output_dir + 'wasting_transition_count.csv').drop(columns='Unnamed: 0')
wasting_transitions.head()

,sex,year,measure,input_draw,scenario,value,x_factor,sq_lns,mam_treatment,sam_treatment,age
0,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,covered,covered,early_neonatal
1,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat1,covered,covered,covered,early_neonatal
2,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,uncovered,covered,early_neonatal
3,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat1,covered,uncovered,covered,early_neonatal
4,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,covered,uncovered,early_neonatal


In [5]:
wasting_transitions.columns

Index(['sex', 'year', 'measure', 'input_draw', 'scenario', 'value', 'x_factor',
       'sq_lns', 'mam_treatment', 'sam_treatment', 'age'],
      dtype='object')

# V&V Step 1: wasting remission rate RRs by treatment coverage

Verify the relative risks for MAM and SAM recovery rates by treatment coverage status in the Wasting transition rate relative risks for wasting treatment table. Note that the relative risks will vary between the baseline and alternative scenarios given the reliance of the relative risks on E_MAM and E_SAM values, which vary between simulation scenarios.

## Step 1.1: calculate target RRs

In [7]:
# artifact
from vivarium import Artifact
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start == 2020', f'age_end <= 5'])

In [10]:
[k for k in art.keys if 'cause' not in k and 'population.' not in k and 'birth' not in k]

['metadata.keyspace',
 'metadata.locations',
 'sequela.moderate_acute_malnutrition.disability_weight',
 'sequela.severe_acute_malnutrition.disability_weight',
 'risk_factor.child_wasting.distribution',
 'alternative_risk_factor.child_wasting.distribution',
 'risk_factor.child_wasting.categories',
 'risk_factor.child_wasting.exposure',
 'risk_factor.child_wasting.relative_risk',
 'risk_factor.child_wasting.population_attributable_fraction',
 'risk_factor.child_stunting.distribution',
 'alternative_risk_factor.child_stunting.distribution',
 'risk_factor.child_stunting.categories',
 'risk_factor.child_stunting.exposure',
 'risk_factor.child_stunting.relative_risk',
 'risk_factor.child_stunting.population_attributable_fraction',
 'risk_factor.severe_acute_malnutrition_treatment.exposure',
 'risk_factor.severe_acute_malnutrition_treatment.distribution',
 'risk_factor.severe_acute_malnutrition_treatment.categories',
 'risk_factor.severe_acute_malnutrition_treatment.relative_risk',
 'risk_fac

In [16]:
sam_tx_rr = art.load('risk_factor.severe_acute_malnutrition_treatment.relative_risk').reset_index()
pd.DataFrame(sam_tx_rr.loc[(sam_tx_rr.sex=='Female')&(sam_tx_rr.age_start >1)]
             .set_index([r for r in sam_tx_rr.columns if 'draw' not in r]).mean(axis=1))
# cat1 = untreated
# cat2 = baseline treatment
# cat3 = alternative treatment

0
sex    age_start age_end year_start year_end affected_entity                                    affected_measure parameter          
Female 2.0       5.0     2020       2021     severe_acute_malnutrition_to_mild_child_wasting    transition_rate  cat1       0.000000
                                                                                                                 cat2       1.000000
                                                                                                                 cat3       1.071584
                                             severe_acute_malnutrition_to_moderate_acute_mal... transition_rate  cat1       3.384485
                                                                                                                 cat2       1.000000
                                                                                                                 cat3       0.846121

In [17]:
sam_tx_rr = art.load('risk_factor.moderate_acute_malnutrition_treatment.relative_risk').reset_index()
pd.DataFrame(sam_tx_rr.loc[(sam_tx_rr.sex=='Female')&(sam_tx_rr.age_start >1)]
             .set_index([r for r in sam_tx_rr.columns if 'draw' not in r]).mean(axis=1))
# cat1 = untreated
# cat2 = baseline treatment
# cat3 = alternative treatment

0
sex    age_start age_end year_start year_end affected_entity                                   affected_measure parameter          
Female 2.0       5.0     2020       2021     moderate_acute_malnutrition_to_mild_child_wasting transition_rate  cat1       0.722711
                                                                                                                cat2       1.000000
                                                                                                                cat3       1.007508

In [35]:
sam_tx_rr = art.load('risk_factor.severe_acute_malnutrition_treatment.relative_risk').reset_index()
mam_tx_rr = art.load('risk_factor.moderate_acute_malnutrition_treatment.relative_risk').reset_index()
tx_rrs = pd.concat([sam_tx_rr, mam_tx_rr])
tx_rrs = tx_rrs.loc[(tx_rrs.age_start == 0.5)&(tx_rrs.sex=='Female')].drop(columns=[c for c in tx_rrs.columns if 'draw' not in c and c != 'affected_entity' and c != 'parameter'])
tx_rrs_baseline = tx_rrs.loc[tx_rrs.parameter=='cat1'].drop(columns='parameter').set_index([c for c in tx_rrs.columns if 'draw' not in c and c != 'parameter'])
#tx_rrs_cat2 = tx_rrs.loc[tx_rrs.parameter=='cat2']
tx_rrs_alternative = tx_rrs_baseline / tx_rrs.loc[tx_rrs.parameter=='cat3'].drop(columns='parameter').set_index([c for c in tx_rrs.columns if 'draw' not in c and c != 'parameter'])
tx_rrs_baseline

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
affected_entity,,,,,,,,,,,,,,,,,,,,,
severe_acute_malnutrition_to_mild_child_wasting,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
severe_acute_malnutrition_to_moderate_acute_malnutrition,3.327213,4.018068,3.362859,2.961939,2.923948,3.948199,3.022492,3.327790,3.977514,3.292555,...,3.317044,3.236318,2.892277,3.021562,2.847928,3.163777,4.588717,2.982359,2.789703,3.018085
moderate_acute_malnutrition_to_mild_child_wasting,0.725869,0.702035,0.709471,0.716635,0.713935,0.728643,0.745621,0.718438,0.701801,0.709031,...,0.704343,0.758650,0.716917,0.729339,0.721335,0.697261,0.733517,0.742891,0.695907,0.709688


In [42]:
pd.DataFrame(tx_rrs_baseline.mean(axis=1))

,0
affected_entity,
severe_acute_malnutrition_to_mild_child_wasting,0.000000
severe_acute_malnutrition_to_moderate_acute_malnutrition,3.384485
moderate_acute_malnutrition_to_mild_child_wasting,0.722711


In [36]:
tx_rrs_alternative

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
affected_entity,,,,,,,,,,,,,,,,,,,,,
severe_acute_malnutrition_to_mild_child_wasting,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
severe_acute_malnutrition_to_moderate_acute_malnutrition,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
moderate_acute_malnutrition_to_mild_child_wasting,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,...,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325


## Step 1.2: calculate simulation remission RRs

In [38]:
def calculate_wasting_transition_rrs(wasting_transitions, wasting_pt,
                                      ages, transition, source_state, 
                                      groupby_cols):
    transition = (wasting_transitions.loc[(wasting_transitions.age.isin(ages))
                                         & (wasting_transitions.measure==transition)]
                  .groupby(groupby_cols).sum())
    pt = (wasting_pt.loc[(wasting_pt.age.isin(ages))
                       & (wasting_pt.cause==source_state)]
          .groupby(groupby_cols).sum())
    transition_rate = transition / pt
    transition_rate_covered = (transition_rate.reset_index().loc[transition_rate.reset_index()[[c for c in groupby_cols if 'treatment' in c][0]]=='covered']
                               .set_index([c for c in groupby_cols if c != [c for c in groupby_cols if 'treatment' in c][0]])
                               .drop(columns=[c for c in groupby_cols if 'treatment' in c][0]))
    rr = transition_rate / transition_rate_covered
    rr = (rr.groupby([c for c in groupby_cols if c != 'input_draw'])
                       .describe(percentiles=[0.025,0.975]))['value']
    return rr

In [39]:
ages = ['6-11_months', '12_to_23_months', '2_to_4']

# mam to mild (r3)
print('MAM to mild RR target baseline:', rr_mam_to_mild_baseline,
      '\nMAM to mild RR target alternative:', rr_mam_to_mild_alternative)
p = calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                 'moderate_acute_malnutrition', 
                                  ['input_draw','scenario','mam_treatment']).reset_index()
p

MAM to mild RR target baseline: 0.7224991121832943 
MAM to mild RR target alternative: 0.7173252279635259


,scenario,mam_treatment,count,mean,std,min,2.5%,50%,97.5%,max
0,baseline,covered,12.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,baseline,uncovered,12.0,0.732549,0.022108,0.695236,0.698051,0.732792,0.767439,0.768510
2,sqlns,covered,12.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,sqlns,uncovered,12.0,0.722863,0.007948,0.711234,0.712130,0.722243,0.738025,0.739468
4,wasting_treatment,covered,12.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,wasting_treatment,uncovered,12.0,0.722315,0.007215,0.711816,0.712513,0.722229,0.735842,0.737586


In [10]:
# sam to mam (r2)
print('SAM to MAM RR target baseline:', rr_sam_to_mam_baseline,
      '\nSAM to MAM RR target alternative:', rr_sam_to_mam_alternative)
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                 'severe_acute_malnutrition', 
                                  ['input_draw','scenario','sam_treatment'])

SAM to MAM RR target baseline: 3.333333333333333 
SAM to MAM RR target alternative: 3.6363636363636362


count      mean       std       min  \
scenario          sam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  3.459278  0.379928  2.903834   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  3.701872  0.226830  3.364422   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  3.713336  0.222868  3.383590   

                                     2.5%       50%     97.5%       max  
scenario          sam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      2.962210  3.444246  4.049090  4.095886  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      3.400713  3.709955  4.072738  4.107935  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      3.418062  3.709263  4.074729  4.103638

In [11]:
# sam to mild (t1)
print('SAM to mild RR target baseline:', rr_sam_to_mild_baseline,
      '\nSAM to mild RR target alternative:', rr_sam_to_mild_alternative)
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                 'severe_acute_malnutrition', 
                                  ['input_draw','scenario','sam_treatment'])

SAM to mild RR target baseline: 0 
SAM to mild RR target alternative: 0


count  mean  std  min  2.5%  50%  97.5%  max
scenario          sam_treatment                                              
baseline          covered         12.0   1.0  0.0  1.0   1.0  1.0    1.0  1.0
                  uncovered       12.0   0.0  0.0  0.0   0.0  0.0    0.0  0.0
sqlns             covered         12.0   1.0  0.0  1.0   1.0  1.0    1.0  1.0
                  uncovered       12.0   0.0  0.0  0.0   0.0  0.0    0.0  0.0
wasting_treatment covered         12.0   1.0  0.0  1.0   1.0  1.0    1.0  1.0
                  uncovered       12.0   0.0  0.0  0.0   0.0  0.0    0.0  0.0

# Step 2: verify that wasting incidence rates do not vary by treatment coverage

(note: this is a validation strategy we had imposed, but should perhaps be rethought?)

In [12]:
# i3 stratified by mam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                 'susceptible_to_child_wasting', 
                                  ['input_draw','scenario','mam_treatment'])

count      mean       std       min  \
scenario          mam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.999939  0.002289  0.994854   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.999734  0.002661  0.995948   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.999804  0.002599  0.996250   

                                     2.5%       50%     97.5%       max  
scenario          mam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.995477  1.000185  1.003138  1.003576  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.996008  0.999242  1.004148  1.004510  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.996351  0.999436  1.004041  1.004255

In [13]:
# i3 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                 'susceptible_to_child_wasting', 
                                  ['input_draw','scenario','sam_treatment'])

count      mean       std       min  \
scenario          sam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.000702  0.001473  0.997788   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.001045  0.002076  0.997175   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.001077  0.002053  0.997099   

                                     2.5%       50%     97.5%       max  
scenario          sam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.998209  1.000512  1.002787  1.002858  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.997611  1.001362  1.003955  1.004368  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.997528  1.001660  1.003905  1.004240

In [14]:
# i2 stratified by mam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                 'mild_child_wasting', 
                                  ['input_draw','scenario','mam_treatment'])

count      mean       std       min  \
scenario          mam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.001503  0.005102  0.991812   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.068621  0.010499  1.052385   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.007020  0.004831  1.001201   

                                     2.5%       50%     97.5%       max  
scenario          mam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.993541  1.001312  1.010794  1.011697  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.053186  1.068011  1.085293  1.087025  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.001384  1.005754  1.015214  1.015929

In [15]:
# i2 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                 'mild_child_wasting', 
                                  ['input_draw','scenario','sam_treatment'])

count      mean       std       min  \
scenario          sam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.002113  0.003775  0.996307   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.028363  0.008115  1.019213   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.004536  0.003475  0.998160   

                                     2.5%       50%     97.5%       max  
scenario          sam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.996840  1.000903  1.008451  1.008472  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.019309  1.027121  1.042047  1.042698  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.998895  1.005650  1.009376  1.009737

In [16]:
# i1 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                 'moderate_acute_malnutrition', 
                                  ['input_draw','scenario','mam_treatment'])

count      mean       std       min  \
scenario          mam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.997987  0.006971  0.987955   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.009715  0.006790  1.001435   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.011713  0.006907  1.002973   

                                     2.5%       50%     97.5%       max  
scenario          mam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.988726  0.998973  1.008985  1.009968  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.001620  1.009200  1.022025  1.023962  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.003100  1.011096  1.024340  1.026263

# Step 3: Verifiy remission rates

In [17]:
# artifact
from vivarium import Artifact
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start == 2020', f'age_end <= 5'])

In [18]:
def calculate_wasting_transition_rates(wasting_transitions, wasting_pt,
                                      ages, transition, source_state, 
                                      groupby_cols, time_to_response=False):
    transition = (wasting_transitions.loc[(wasting_transitions.age.isin(ages))
                                         & (wasting_transitions.measure==transition)]
                  .groupby(groupby_cols).sum())
    pt = (wasting_pt.loc[(wasting_pt.age.isin(ages))
                       & (wasting_pt.cause==source_state)]
          .groupby(groupby_cols).sum())
    transition_rate = transition / pt
    if time_to_response==True:
        transition_rate = 365 / transition_rate
    transition_rate = (transition_rate.groupby([c for c in groupby_cols if c != 'input_draw'])
                       .describe(percentiles=[0.025,0.975]))['value']
    return transition_rate

In [19]:
#r4 (30 days)
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_susceptible_to_child_wasting_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario'], time_to_response=True)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,29.721623,0.026310,29.683731,29.684603,29.730404,29.757882,29.762796
sqlns,12.0,29.718298,0.028569,29.673875,29.675521,29.727446,29.757033,29.761452
wasting_treatment,12.0,29.719767,0.026890,29.679180,29.681159,29.726658,29.756436,29.760920


In [20]:
# r3 
# blend of 63 days untreated and 41 days treated
# looks reasonable
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'], time_to_response=True)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,60.012739,0.750323,58.804160,58.909057,59.863399,61.239740,61.297961
sqlns,12.0,54.890663,0.480334,54.335579,54.340788,54.928611,55.639804,55.680703
wasting_treatment,12.0,54.655840,0.453651,54.049411,54.079609,54.700610,55.322488,55.341138


In [21]:
# r3
# blend of 63 days untreated and 41 days treated
# looks reasonable
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','mam_treatment'], time_to_response=True)

count       mean       std        min  \
scenario          mam_treatment                                          
baseline          covered         12.0  46.271819  1.403786  43.914885   
                  uncovered       12.0  63.165286  0.130155  62.869995   
sqlns             covered         12.0  45.683747  0.507614  45.010550   
                  uncovered       12.0  63.198775  0.256555  62.847392   
wasting_treatment covered         12.0  45.660454  0.462132  45.029353   
                  uncovered       12.0  63.214367  0.233382  62.884071   

                                      2.5%        50%      97.5%        max  
scenario          mam_treatment                                              
baseline          covered        44.080250  46.291173  48.412997  48.449715  
                  uncovered      62.920732  63.160993  63.361029  63.364654  
sqlns             covered        45.026003  45.584502  46.556504  46.587927  
                  uncovered      62.874748  63.190961  63.596355  63.615528  
wasting_treatment covered        45.036218  45.613430  46.427653  46.444812  
                  uncovered      62.907792  63.196797  63.525018  63.528925

In [22]:
# r2
# untreated SAM recovery to MAM 
print('Untreated target should be ~> 54\nBased on uncovered rate...\ncovered baseline target: ', 365/((365/57.5) * 0.3),
      '\ncovered alternative target: ',  365/((365/57.5) * 0.275))
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                   'severe_acute_malnutrition', 
                                      ['input_draw','scenario','sam_treatment'], time_to_response=True)


Untreated target should be ~> 54
Based on uncovered rate...
covered baseline target:  191.66666666666669 
covered alternative target:  209.0909090909091


count        mean        std         min  \
scenario          sam_treatment                                             
baseline          covered         12.0  199.556619  45.416246  141.984182   
                  uncovered       12.0   57.483131  10.259483   44.332889   
sqlns             covered         12.0  213.212954  42.348175  164.838898   
                  uncovered       12.0   57.500178  10.289377   44.320591   
wasting_treatment covered         12.0  213.783060  42.158830  165.803223   
                  uncovered       12.0   57.487897  10.298092   44.257539   

                                       2.5%         50%       97.5%  \
scenario          sam_treatment                                       
baseline          covered        145.000375  199.601789  283.653723   
                  uncovered       45.407645   54.147887   76.795328   
sqlns             covered        165.088518  210.781956  293.302138   
                  uncovered       45.429033   54.191544   76.999341   
wasting_treatment covered        165.858132  211.394668  293.508627   
                  uncovered       45.389061   54.129568   76.930967   

                                        max  
scenario          sam_treatment              
baseline          covered        299.357745  
                  uncovered       80.448411  
sqlns             covered        308.705589  
                  uncovered       80.753293  
wasting_treatment covered        308.787212  
                  uncovered       80.668561

In [23]:
# t1 
# time to recovery effectively treated SAM = 48.3
print('covered baseline target: ', 365 / (365 / 48.3 * 0.7),
      '\ncovered alternative target: ', 365 / (365 / 48.3 * 0.725))

calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'severe_acute_malnutrition', 
                                      ['input_draw','scenario','sam_treatment'], time_to_response=True)

covered baseline target:  69.0 
covered alternative target:  66.62068965517241


count       mean       std        min  \
scenario          sam_treatment                                          
baseline          covered         12.0  68.591715  3.044725  64.845801   
                  uncovered       12.0        inf       NaN        inf   
sqlns             covered         12.0  66.520794  1.428210  64.530287   
                  uncovered       12.0        inf       NaN        inf   
wasting_treatment covered         12.0  66.470635  1.368444  64.518981   
                  uncovered       12.0        inf       NaN        inf   

                                      2.5%        50%      97.5%        max  
scenario          sam_treatment                                              
baseline          covered        64.921415  67.991928  73.462761  74.247180  
                  uncovered            inf        inf        inf        inf  
sqlns             covered        64.636327  66.236236  68.808434  69.133622  
                  uncovered            inf        inf        inf        inf  
wasting_treatment covered        64.617716  66.042653  68.608341  68.897186  
                  uncovered            inf        inf        inf        inf

# Step 4: validate incidence rates

In [24]:
#i3 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                   'susceptible_to_child_wasting', 
                                      ['input_draw','scenario'], time_to_response=True)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,101.989802,0.758337,99.908964,100.366012,102.192207,102.803435,102.924144
sqlns,12.0,101.971766,0.757605,99.887643,100.341031,102.172114,102.773805,102.893710
wasting_treatment,12.0,101.977830,0.758406,99.895221,100.349707,102.182948,102.790027,102.913304


In [25]:
#i2
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario'], time_to_response=True)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,160.486655,4.402687,155.016590,155.033648,159.706739,168.927192,169.484239
sqlns,12.0,174.063622,4.526199,170.165141,170.215190,172.113476,183.284899,184.242359
wasting_treatment,12.0,159.955161,4.340365,154.622205,154.642501,159.247063,168.290734,168.851250


In [26]:
#i2
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'], time_to_response=True)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,195.433992,17.306117,166.658496,170.506081,191.543278,221.987080,226.479743
sqlns,12.0,194.113221,17.207914,165.126728,169.043768,190.556575,220.553777,225.026847
wasting_treatment,12.0,194.826187,17.175172,166.125089,169.942300,191.270570,221.310176,225.806979


In [27]:
# overall transition out of SAM
print('baseline target: 6.7 (95% CI: 5.3-8.4)')
overall_sam = (wasting_transitions.loc[(wasting_transitions.measure.str.contains('severe_acute_malnutrition_to'))
                       & (wasting_transitions.age.isin(ages))]
               .groupby(['input_draw','scenario']).sum()).drop(columns='year')
deaths = pd.read_csv(output_dir + 'deaths.csv')
sam_deaths = deaths.loc[(deaths.wasting_state=='severe_acute_malnutrition')
                       &(deaths.age.isin(ages))].groupby(['input_draw','scenario']).sum()
sam_pt = (wasting_pt.loc[(wasting_pt.cause=='severe_acute_malnutrition')
                        & (wasting_pt.age.isin(ages))]
         .groupby(['input_draw','scenario']).sum())#['value']
overall_sam_remission = (((overall_sam + sam_deaths)/ sam_pt)).groupby('scenario').describe(percentiles=[0.025,0.975])['value']
overall_sam_remission

baseline target: 6.7 (95% CI: 5.3-8.4)


,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,6.876026,0.687064,5.533690,5.721291,6.991568,7.855919,7.961835
sqlns,12.0,6.959922,0.607378,5.797439,5.953345,7.072093,7.829758,7.921951
wasting_treatment,12.0,6.963337,0.604282,5.812116,5.965293,7.084688,7.833892,7.933162


In [28]:
365 / 6.7

54.47761194029851

In [29]:
365 / 6.87

53.12954876273653

# look at incidence rates stratified by x-factor exposure

In [30]:
#i3 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                   'susceptible_to_child_wasting', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

count      mean       std       min      2.5%  \
scenario          x_factor                                                  
baseline          cat1       12.0  3.572317  0.026351  3.546107  3.548330   
                  cat2       12.0  3.584784  0.027905  3.546469  3.552122   
sqlns             cat1       12.0  3.573607  0.026298  3.547540  3.549600   
                  cat2       12.0  3.584929  0.027881  3.547182  3.552795   
wasting_treatment cat1       12.0  3.573134  0.026388  3.546130  3.548747   
                  cat2       12.0  3.584915  0.027839  3.547152  3.552733   

                                 50%     97.5%       max  
scenario          x_factor                                
baseline          cat1      3.566871  3.629432  3.647225  
                  cat2      3.578696  3.644114  3.658652  
sqlns             cat1      3.567867  3.630489  3.648380  
                  cat2      3.578727  3.644454  3.659185  
wasting_treatment cat1      3.567247  3.630180  3.648210  
                  cat2      3.578685  3.644282  3.658779

In [31]:
#i2 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

count      mean       std       min      2.5%  \
scenario          x_factor                                                  
baseline          cat1       12.0  3.501611  0.093468  3.308654  3.321715   
                  cat2       12.0  1.106286  0.030013  1.047732  1.052163   
sqlns             cat1       12.0  3.214266  0.083252  3.029249  3.047006   
                  cat2       12.0  1.014440  0.025445  0.958660  0.964321   
wasting_treatment cat1       12.0  3.503163  0.093480  3.310540  3.323202   
                  cat2       12.0  1.106623  0.029994  1.048007  1.052401   

                                 50%     97.5%       max  
scenario          x_factor                                
baseline          cat1      3.513056  3.620910  3.621660  
                  cat2      1.106123  1.145240  1.146727  
sqlns             cat1      3.252355  3.284248  3.285309  
                  cat2      1.022157  1.039790  1.040039  
wasting_treatment cat1      3.513490  3.621671  3.622941  
                  cat2      1.106970  1.145438  1.146793

In [32]:
#i1
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

count      mean       std       min      2.5%  \
scenario          x_factor                                                  
baseline          cat1       12.0  2.263399  0.200477  1.939847  1.980745   
                  cat2       12.0  0.696571  0.060926  0.600500  0.610332   
sqlns             cat1       12.0  2.273005  0.202231  1.946346  1.987781   
                  cat2       12.0  0.697375  0.061679  0.598481  0.608116   
wasting_treatment cat1       12.0  2.268927  0.200079  1.944074  1.985067   
                  cat2       12.0  0.697917  0.061723  0.600144  0.610025   

                                 50%     97.5%       max  
scenario          x_factor                                
baseline          cat1      2.290835  2.581116  2.642229  
                  cat2      0.704319  0.797671  0.809668  
sqlns             cat1      2.296712  2.595219  2.657701  
                  cat2      0.705079  0.798211  0.809414  
wasting_treatment cat1      2.292664  2.586029  2.646818  
                  cat2      0.707057  0.799459  0.811886

In [33]:
#i1
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor','mam_treatment'], False)

count      mean       std       min  \
scenario          x_factor mam_treatment                                        
baseline          cat1     covered         12.0  2.265123  0.191186  1.954386   
                           uncovered       12.0  2.263212  0.202294  1.937737   
                  cat2     covered         12.0  0.703675  0.074798  0.573828   
                           uncovered       12.0  0.695927  0.059940  0.604367   
sqlns             cat1     covered         12.0  2.276235  0.206829  1.933188   
                           uncovered       12.0  2.270785  0.199354  1.954470   
                  cat2     covered         12.0  0.696421  0.060782  0.612083   
                           uncovered       12.0  0.698263  0.064101  0.589501   
wasting_treatment cat1     covered         12.0  2.270331  0.203058  1.932388   
                           uncovered       12.0  2.267844  0.198217  1.951663   
                  cat2     covered         12.0  0.697082  0.061176  0.605211   
                           uncovered       12.0  0.698772  0.063378  0.596600   

                                              2.5%       50%     97.5%  \
scenario          x_factor mam_treatment                                 
baseline          cat1     covered        1.994249  2.283676  2.569951   
                           uncovered      1.978678  2.292346  2.584292   
                  cat2     covered        0.584010  0.708922  0.806406   
                           uncovered      0.612704  0.698820  0.796244   
sqlns             cat1     covered        1.976132  2.302583  2.611025   
                           uncovered      1.992736  2.293161  2.585700   
                  cat2     covered        0.618370  0.699918  0.786517   
                           uncovered      0.600975  0.699650  0.807033   
wasting_treatment cat1     covered        1.973538  2.295573  2.594931   
                           uncovered      1.990395  2.290701  2.582025   
                  cat2     covered        0.613409  0.707619  0.790401   
                           uncovered      0.607677  0.697339  0.806701   

                                               max  
scenario          x_factor mam_treatment            
baseline          cat1     covered        2.627717  
                           uncovered      2.645632  
                  cat2     covered        0.807606  
                           uncovered      0.810151  
sqlns             cat1     covered        2.676762  
                           uncovered      2.644258  
                  cat2     covered        0.793073  
                           uncovered      0.821729  
wasting_treatment cat1     covered        2.656472  
                           uncovered      2.639589  
                  cat2     covered        0.800486  
                           uncovered      0.821064